# Przygotowanie zbioru shs100k

Przygotowanie zbioru shs100k na potrzeby treningu dla większego zbioru danych. Główną optymalizacją wydajności jest usunięcie bardzo długich piosenek, w których często znajdują się momenty ciszy. Następnie wszystkie piosenki zostały przycięte do długości 2 minut.
Dzięki tym optymalizacjom udało się dokonać treningu modelu na zbiorze 8000 utworów.

In [8]:
pip install yt-dlp pandas ffmpeg-python

In [ ]:
import os
import csv
import json
import yt_dlp
import whisper
from tqdm import tqdm
from collections import defaultdict
import subprocess

In [ ]:
import os
import pandas as pd
import subprocess
from yt_dlp import YoutubeDL

OUTPUT_DIR = "downloads"
MAX_DURATION_SEC = 300
TRIM_DURATION = 120

df = pd.read_csv("shs100k.csv")

ydl_opts = {
    'format': 'bestaudio/best',
    'quiet': True,
    'noplaylist': True,
    'outtmpl': '%(id)s.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }]
}

def download_and_trim(row):
    url = f"https://www.youtube.com/watch?v={row['Video ID']}"
    clique_folder = os.path.join(OUTPUT_DIR, f"clique_{row['clique']}")
    os.makedirs(clique_folder, exist_ok=True)

    with YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(url, download=False)
            duration = info.get("duration", 0)

            if duration > MAX_DURATION_SEC:
                print(f"Pomijam {row['title']} - za długie: {duration}s")
                return

            print(f"Pobieram: {row['title']} ({duration}s)")
            ydl.download([url])

            input_file = f"{info['id']}.mp3"
            output_file = os.path.join(clique_folder, f"{row['performer']}_{info['id']}.mp3")

            subprocess.run([
                "ffmpeg", "-y", "-i", input_file,
                "-t", str(TRIM_DURATION),
                "-acodec", "copy", output_file
            ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

            os.remove(input_file)

        except Exception as e:
            print(f"Błąd przy {row['title']} - {e}")

for _, row in df.iterrows():
    download_and_trim(row)
